In [2]:
import hopsy
from PolyRound.api import PolyRoundApi
import os
import time
import numpy as np
import copy

In [3]:
model_path = os.path.join("hopsy","examples","test_data", "e_coli_core.xml")
polytope = PolyRoundApi.sbml_to_polytope(model_path)

Restricted license - for non-production use only - expires 2025-11-24


In [4]:
problem = hopsy.Problem(polytope.A, polytope.b)
problem = hopsy.add_box_constraints(problem, upper_bound=10_000, lower_bound=-10_000, simplify=True)
start = time.perf_counter()
problem = hopsy.round(problem)
print("Computing rounding transformation took", time.perf_counter()-start,"seconds")

Computing rounding transformation took 2.1488025359999483 seconds


In [5]:
print(polytope.A.shape)
print(polytope.b.shape)
print(problem.A.shape)
print(problem.b.shape)

(190, 95)
(190,)
(190, 95)
(190,)


In [6]:
seed = 511
chains, rngs = hopsy.setup(problem, seed, n_chains=4)
n_samples = 100_000
# Either use thinning rule, see  10.1371/journal.pcbi.1011378
# or use one-shot transformation (for expert users). We show one-shot transformation at the end.
thinning = int(1./6*problem.transformation.shape[1])

<h3> Sampling with Transform <h3>

In [7]:
%%script False
start = time.perf_counter()
accrate, samples = hopsy.sample(chains, rngs, n_samples, thinning=thinning, n_procs=4)
# accrate is 1 for uniform samples with the default chains given by hopsy.setup()
print("sampling with internal trafo took", time.perf_counter()-start,"seconds")
print(samples.shape)
rhat = np.max(hopsy.rhat(samples))
print("rhat:", rhat)
ess = np.min(hopsy.ess(samples)) / len(chains)
print("ess:", ess)

Couldn't find program: 'False'


<h3>Sampling without Transform<h3>

In [8]:
assert problem.transformation is not None
# deep copy enures that we do not edit the original problem
problem2 = copy.deepcopy(problem)
problem2.transformation=None
problem2.shift=None
seed = 512
chains, rngs = hopsy.setup(problem2, seed, n_chains=4)
# thinning is still advised when hard drive memory is limisted to not to store too many samples 
thinning = int(1./6*problem.A.shape[1])  

start = time.perf_counter()
accrate, sample_stack = hopsy.sample(chains, rngs, n_samples, thinning=thinning, n_procs=4)
# accrate is 1 for uniform samples with the default chains given by hopsy.setup()
print("sampling took", time.perf_counter()-start,"seconds")




sampling took 1.349342826000111 seconds


In [9]:
%%script False
print('sample shape', sample_stack.shape)
rhat = np.max(hopsy.rhat(sample_stack))
print("rhat:", rhat)
ess = np.min(hopsy.ess(sample_stack)) / len(chains)
print("ess:", ess)

Couldn't find program: 'False'


In [10]:
# Make copy of sample_stack
sample_stack_seq = sample_stack.copy()
sample_stack_pl = sample_stack.copy()

In [11]:
print(sample_stack.dtype)

float64


<h3>Transform Back: Sequential<h3>

In [12]:
# transform samples back all at once
shift_t = np.array([problem.shift]).T
start_trafo = time.perf_counter()
full_samples = np.zeros((len(chains), n_samples, sample_stack.shape[2]))
for i in range(len(chains)):
    full_samples[i] = (problem.transformation@sample_stack[i].T).T + np.tile(shift_t, (1, n_samples)).T
    
print("transformation took", time.perf_counter()-start_trafo,"seconds")
print('sample stats are the same (save numerics) before and after the linear transformation:')


transformation took 0.46120086100017943 seconds
sample stats are the same (save numerics) before and after the linear transformation:


In [13]:
%%script False
rhat = np.max(hopsy.rhat(full_samples))
print("rhat:", rhat)
ess = np.min(hopsy.ess(full_samples)) / len(chains)
print("ess:", ess)

Couldn't find program: 'False'


<h2>Transform Back: Parallel<h2>

In [18]:
from numba import cuda
from numba import *
import math

In [15]:
# Device Properties to get execution config
# Get the current device
device = cuda.get_current_device()

# Access device properties
warp_size = device.WARP_SIZE
multi_processor_count = device.MULTIPROCESSOR_COUNT

# print(warp_size)
# print(multi_processor_count)
tpb = warp_size                     # threads per block
nb = multi_processor_count * 32     # number of blocks
print(tpb)
print(nb)

32
512


In [16]:
# This version needs A and b to be input as transposed.
"""
Perform Y = X.A + b
X : (n_samples, A.shape[0]), n_samples is parallelised
Y : same shape as X
b : Row vector
"""
@cuda.jit
def tranformv1(A:np.ndarray, b: np.ndarray, X: np.ndarray, Y: np.ndarray):
    idx = cuda.grid(1)
    stride = cuda.gridsize(1)
    for i in range(idx,X.shape[0],stride):
        for j in range(A.shape[1]):
            temp = 0
            for k in range(A.shape[0]):
                temp += X[i,k] * A[k,j]
            Y[i,j] = temp + b[0,j]

In [17]:
tpb2d = tpb
@cuda.jit
def transformv2(A:np.ndarray, b: np.ndarray, X: np.ndarray, Y: np.ndarray):
    # tpb2d, _ = cuda.blockDim(2)

    sX = cuda.shared.array(shape=(tpb2d,tpb2d), dtype=float64)
    sA = cuda.shared.array(shape=(tpb2d,tpb2d), dtype=float64)

    x, y = cuda.grid(2)
    # stridex, stridey = cuda.gridsize(2)

    tx, ty = cuda.threadIdx.x, cuda.threadIdx.y 

    bpg = cuda.gridDim.x    # blocks per grid, aka grid Dim

    
    # for i in range(idx,Y.shape[0],stridex):
    #     for j in range(idy, Y.shape[1],stridey):
    temp = float64(0.)
    for i in range(bpg):
        
        # Preload chunks of data into shared memory
        sX[tx,ty] = 0 
        sA[tx,ty] = 0
        if y < X.shape[0] and (tx + i * tpb2d) < X.shape[1]:
            sX[ty,tx] = X[y,tx + i * tpb2d]
        if x < A.shape[1] and (ty + i * tpb2d) < A.shape[0]:
            sA[ty, tx] = A[ty + i * tpb2d, x]

        # wait till loading complete
        cuda.syncthreads()

        # Do partial row * col
        for j in range(tpb2d):
            temp += sX[ty, j] * sA[j, tx]

        # Sync again
        cuda.syncthreads()

    # Put result back in
    if y < Y.shape[0] and x < Y.shape[1]:
        Y[y,x] = temp + b[0,x]

In [15]:
%%script False
shift_t_pl = np.array([problem.shift])
start_trafo = time.perf_counter()
full_samples_pl = np.zeros((len(chains), n_samples, sample_stack.shape[2]))

# Device arrays
d_transformation = cuda.to_device(problem.transformation.T)
d_sample_stack = cuda.to_device(sample_stack)
d_shift = cuda.to_device(shift_t_pl)
d_full_samples = cuda.to_device(full_samples_pl)
for i in range(len(chains)):
    tranformv1[nb,tpb](d_transformation,
               d_shift,
               d_sample_stack[i],
               d_full_samples[i])
full_samples_pl = d_full_samples.copy_to_host()
print("transformation took", time.perf_counter()-start_trafo,"seconds")

Couldn't find program: 'False'


With Streams

In [17]:
shift_t_pl = np.array([problem.shift])
start_trafo = time.perf_counter()
full_samples_pl = np.zeros((len(chains), n_samples, sample_stack.shape[2]))

# Device arrays
d_transformation = cuda.to_device(problem.transformation.T)
d_sample_stack = cuda.to_device(sample_stack)
d_shift = cuda.to_device(shift_t_pl)
d_full_samples = cuda.to_device(full_samples_pl)
streams = [cuda.stream() for _ in range(len(chains))]
for i in range(len(chains)):
    tranformv1[nb,tpb,streams[i]](d_transformation,
               d_shift,
               d_sample_stack[i],
               d_full_samples[i])
full_samples_pl = d_full_samples.copy_to_host()
print("transformation took", time.perf_counter()-start_trafo,"seconds")

/home/harris/miniconda3/envs/vmcmc/lib/python3.10/site-packages/numba/cuda/cudadrv/devicearray.py:888: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


transformation took 0.7409150810017309 seconds


In [ ]:
%%script False
rhat = np.max(hopsy.rhat(full_samples_pl))
print("rhat:", rhat)
ess = np.min(hopsy.ess(full_samples_pl)) / len(chains)
print("ess:", ess)

In [ ]:
error  = 0
for i in range(len(chains)):
    error += np.linalg.norm(full_samples[i] - full_samples_pl[i],2)

print(error)

In [20]:
# Test Setup
N = 20
A = np.random.random((5,5)).astype(np.float32)
X = np.random.random((N,5)).astype(np.float32)
b = np.random.random((1,5)).astype(np.float32)
Y = np.zeros_like(X).astype(np.float32)

In [25]:
tpb2d_ = (tpb,tpb)
# nb2d_ = (math.ceil(Y.shape[0]/tpb)(Y.s),math.cel)
nb2d_ = (math.ceil(Y.shape[0] / tpb), math.ceil(Y.shape[1]/tpb))

print(tpb2d_)
print(nb2d_)


(32, 32)
(1, 1)


In [26]:
truth = X@A + np.tile(b,(N,1))
btruth = X@A + b

# tranformsl(A,b,X,Y)
dA = cuda.to_device(A)
dX = cuda.to_device(X)
db = cuda.to_device(b)
dY = cuda.to_device(Y)
tranformv1[nb,tpb](dA,db,dX,dY)
# transformv2[nb2d_,tpb2d_](dA,db,dX,dY)
Y = dY.copy_to_host()
